In [ ]:
from playwright.async_api import async_playwright

In [ ]:
async with async_playwright() as p:
    browser = await p.chromium.launch()
    page = await browser.new_page()
    await page.goto("https://www.basketball-reference.com/leagues/NBA_2020_totals.html")
    print(await page.title())
    html = await page.inner_html("#all_totals_stats")

In [ ]:
import pandas as pd

stats = pd.read_html(html)

In [ ]:
players = stats[0]

In [ ]:
summary = players[["Player", "Age", "Tm"]].groupby("Player", group_keys=False).agg(['count', 'min'])

In [ ]:
summary.columns = ["team_count", "age", "tc", "team"]

In [ ]:
del summary["tc"]

In [ ]:
summary["age"] = pd.to_numeric(summary["age"], errors="coerce")
summary = summary.dropna()
summary = summary[(summary["team_count"] == 1) & (summary["age"] > 25)]

In [ ]:
valid_players = pd.Series(summary.index)
valid_players = list(valid_players.apply(lambda x: ", ".join(reversed(x.split(" ")))))
summary["folder_name"] = valid_players

In [ ]:
summary["team_number"] = summary["team"].astype("category").cat.codes

In [ ]:
summary.to_csv("labels.csv")